In [1]:
from selenium import webdriver
from reviews import Review, ReviewList
from bookmark import Bookmark
from schema import vietnam_schema
import random
import json

import time
from bs4 import BeautifulSoup as bs

fname = '../tests/test_files/review1.html'
with open(fname,'r') as f:
    html = f.read()
soup = bs(html)

In [2]:
class PageCrawler(object):

    def __init__(self, driver):
        self.driver = driver
        self.base_url = 'http://www.tripadvisor.com'
        self.url = None
        self.bookmarker = None
        self.pagetype = None
        self.attrs = {}
        
    def update_bookmarker(self, new_pos_dict):
        self.bookmarker.update(new_pos_dict)

    def update_attr(self, new_attrs):
        self.attrs.update(new_attrs)
    
    def listfct(self):
        pass
    
    def nextfct(self):
        pass
    
    def execfct(self):
        pass
    
    def start(self):
        while self.url: 
            self.driver.get(self.url)
            time.sleep(random.randint(5,10))
            html = self.driver.page_source
            soup = bs(html)
            div_list = self.listfct(soup)
            for item in div_list[:3]:
                self.execfct(item)
            self.update_bookmarker({self.pagetype:self.url})
            self.url = self.nextfct(soup)
         

In [3]:
class ReviewCrawler(PageCrawler):
    def __init__(self, driver):
        super(ReviewCrawler, self).__init__(driver)
        self.review_list = ReviewList('test.db', vietnam_schema, 'vietnam')
        self.pagetype = 'review_page'
        
    def listfct(self, soup):        
        tags = soup.find('div',{'id':'REVIEWS'}).findChildren(recursive=False)
        reviews = filter(lambda tag: tag.has_attr('id') and 'review_' in tag['id'], tags)
        return reviews
        
    def nextfct(self, soup):
        navbar = soup.find('div',{'class': "unified pagination "})
        return None
        try:
            return self.base_url + navbar.find(lambda tag: tag.text == 'Next')['href']
        except:
            return None
        
    def execfct(self, review_soup):
        review = Review(review_soup, self.attrs)
        self.review_list.append(review)

In [4]:
class ParentCrawler(PageCrawler):
    def __init__(self, driver):
        super(ParentCrawler, self).__init__(driver)
        self.driver = driver
        self.pagetype = 'parent_page'
#        self.parent_crawler = ParentCrawler(driver)
    
    def init_child(self):
        self.review_crawler = ReviewCrawler(self.driver)
    
    def set_bookmarker(self, x):
        self.bookmarker = x
#        self.review_crawler.bookmarker = self.bookmarker
        self.review_crawler.bookmarker = x
        
    def listfct(self, soup):
        parent_tags = soup.findAll('div',{'class':'property_title'})
        return [(x.a.text, base_url + x.a['href']) for x in parent_tags]
    
    def execfct(self, item):
        self.review_crawler.url = item[1]
        #if location
        self.review_crawler.update_attr(self.attrs)
        self.review_crawler.update_attr({'item_reviewed':item[0]})
        #if group
        self.review_crawler.start()

    def nextfct(self, soup):
        tags = soup.find('div',{'id':'pager_top', 'class':'pgLinks'})
        try:
            next_url = tags.find(lambda tag: tag.name=='a' and tag.text == u"\u00BB")['href']
            return self.base_url + next_url
        except (TypeError, AttributeError):
            return None

In [5]:
with open('cities.json', 'r') as f:
    lines = f.read()
data = json.loads(lines)

#['Country', 'ItemReviewed', 'Type', 'ParentGroup', 'Location']

base_url = "http://www.tripadvisor.com"
profile = webdriver.FirefoxProfile("/Users/alexrhee/Library/Application Support/Firefox/Profiles/Selenium")
driver = webdriver.Firefox(profile)
driver.set_page_load_timeout(15)
crawler = ParentCrawler(driver)
crawler.init_child()

crawler.set_bookmarker(Bookmark('bookmark.json'))

base_attr = {'country':'Vietnam', 'type':'Things to do'}
crawler.update_attr(base_attr)

url = "http://www.tripadvisor.com/Attractions-g303945-Activities-Ninh_Binh_Ninh_Binh_Province.html"
crawler.url = url
crawler.update_attr({'location':'Ninh Binh'})
crawler.start()


<bookmark.Bookmark instance at 0x1094eb248>

In [260]:
#for city in data:
#    crawl.url = base_url + city[1]
#    crawl.update_attr({'Location':city[0]})
#    crawl.start()

f = Bookmark('bookmark.json')
type(f)

instance

In [292]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
